# Gridding Operations

In [1]:
import pandas as pd
import numpy as np
from netCDF4 import Dataset
import h5py
from matplotlib import pyplot as plt
from cartopy import crs as ccrs
import scipy.interpolate

In [4]:
fires = pd.read_csv('../data/VIIRSNDE_global2018312.v1.0.txt')
fires.head()

,Lon,Lat,Mask,Conf,brt_t13(K),frp(MW),line,sample,Sat,YearDay,Hour
0,27.110006,30.769241,8,52,302.877533,5.814295,242,1735,NDE,2018312,1
1,26.083252,30.534357,9,100,332.959717,24.340988,301,1620,NDE,2018312,1
2,34.865997,28.162659,8,38,301.165985,6.107953,396,2589,NDE,2018312,1
3,34.872623,28.161121,8,71,307.277985,9.287819,396,2590,NDE,2018312,1
4,34.865070,28.158880,8,39,301.227783,6.001442,402,2590,NDE,2018312,1


In [5]:
# Number of nx and ny points for the grid. 720 nx, 360 ny creates a 1.0 degree grid
coverage = [-180.0, -90.0, 180.0, 90.0]

grid_size = 1.0

num_points_x = int(coverage[2] - coverage[0] / grid_size)
num_points_y = int(coverage[3] - coverage[1] / grid_size)

print(num_points_x, num_points_y)

360 180


In [7]:
# Using a real step length will skip the end value (exclusive)
print(np.mgrid[0:4:1.0])

[0. 1. 2. 3.]


In [8]:
#Using a complex number of bins will include the end value (inclusvie)
print(np.mgrid[0:4:5j])

[0. 1. 2. 3. 4.]


In [10]:
nx = complex(0, num_points_x)
ny = complex(0, num_points_y)
Xnew, Ynew = np.mgrid[coverage[0]:coverage[2]:nx, coverage[1]:coverage[3]:ny]